# Caesar 암호

In [2]:
key = 'abcdefghijklmnopqrstuvwxyz'
def enc_caesar(n, plaintext):
    result = ''
    for l in plaintext.lower():
        try:
            i = (key.index(l) + n) % 26
            result += key[i]
        except ValueError:
            result += l
    return result.lower()

plaintext = 'YOU ARE IN DANGER'
ciphertext = enc_caesar(3, plaintext)
print (ciphertext)



brx duh lq gdqjhu


# Enigma

In [2]:
class EnigmaMachine:
    def __init__(self, rotors, reflector, plugboard):
        self.rotors = rotors
        self.reflector = reflector
        self.plugboard = plugboard
        self.initial_rotor_positions = [0, 0, 0]
        self.rotor_positions = [0, 0, 0]

    def set_rotor_positions(self, positions):
        self.initial_rotor_positions = positions[:]
        self.rotor_positions = positions[:]

    def reset_rotor_positions(self):
        self.rotor_positions = self.initial_rotor_positions[:]

    def plugboard_swap(self, char):
        return self.plugboard.get(char, char)

    def rotor_encrypt_forward(self, char, rotor, position):
        index = (ord(char) - 65 + position) % 26
        encrypted_char = rotor[index]
        return chr((ord(encrypted_char) - 65 - position + 26) % 26 + 65)

    def rotor_encrypt_backward(self, char, rotor, position):
        index = (ord(char) - 65 + position) % 26
        encrypted_char = chr(rotor.index(chr(index + 65)) + 65)
        return chr((ord(encrypted_char) - 65 - position + 26) % 26 + 65)

    def reflector_swap(self, char):
        index = ord(char) - 65
        return self.reflector[index]

    def step_rotors(self):
        self.rotor_positions[0] = (self.rotor_positions[0] + 1) % 26
        if self.rotor_positions[0] == 0:
            self.rotor_positions[1] = (self.rotor_positions[1] + 1) % 26
            if self.rotor_positions[1] == 0:
                self.rotor_positions[2] = (self.rotor_positions[2] + 1) % 26

    def encrypt_character(self, char, is_first_character=False):
        if char == ' ':
            return ' '
        
        self.step_rotors()
        
        if is_first_character:
            print(f"Initial character: {char}")

        # Plugboard swap
        char = self.plugboard_swap(char)
        if is_first_character:
            print(f"After plugboard swap: {char}")

        # Forward through the rotors
        for i, rotor in enumerate(self.rotors):
            position = self.rotor_positions[i]
            char = self.rotor_encrypt_forward(char, rotor, position)
            if is_first_character:
                print(f"After rotor {i+1} forward: {char}")

        # Reflector swap
        char = self.reflector_swap(char)
        if is_first_character:
            print(f"After reflector swap: {char}")

        # Backward through the rotors
        for i, rotor in enumerate(reversed(self.rotors)):
            position = self.rotor_positions[-(i+1)]
            char = self.rotor_encrypt_backward(char, rotor, position)
            if is_first_character:
                print(f"After rotor {len(self.rotors)-i} backward: {char}")

        # Plugboard swap again
        char = self.plugboard_swap(char)
        if is_first_character:
            print(f"After final plugboard swap: {char}")

        if is_first_character:
            print(f"Encrypted character: {char}")
            print("")

        return char

    def encrypt_message(self, message):
        self.reset_rotor_positions()
        encrypted_message = ''.join(self.encrypt_character(char, is_first_character=(i == 0)) for i, char in enumerate(message.upper()))
        return encrypted_message


# Define the rotors, reflector, and plugboard settings
rotor_1 = "EKMFLGDQVZNTOWYHXUSPAIBRCJ"
rotor_2 = "AJDKSIRUXBLHWTMCQGZNPYFVOE"
rotor_3 = "BDFHJLCPRTXVZNYEIWGAKMUSQO"
reflector_B = "YRUHQSLDPXNGOKMIEBFZCWVJAT"
plugboard_settings = {
    'A': 'B', 'B': 'A',
    'C': 'D', 'D': 'C',
    'E': 'F', 'F': 'E'
    # Add more plugboard connections as needed
}

# Create the Enigma machine
enigma = EnigmaMachine([rotor_1, rotor_2, rotor_3], reflector_B, plugboard_settings)

# Set initial rotor positions
initial_positions = [0, 0, 0]
enigma.set_rotor_positions(initial_positions)

# Encrypt a message
message = "HELLO WORLD"
print("Encrypting message:")
encrypted_message = enigma.encrypt_message(message)
print(f"Original message: {message}")
print(f"Encrypted message: {encrypted_message}")

# Decrypt the message
print("\nDecrypting message:")
enigma.set_rotor_positions(initial_positions)
decrypted_message = enigma.encrypt_message(encrypted_message)
print(f"Encrypted message: {encrypted_message}")
print(f"Decrypted message: {decrypted_message}")


Encrypting message:
Initial character: H
After plugboard swap: H
After rotor 1 forward: U
After rotor 2 forward: P
After rotor 3 forward: E
After reflector swap: Q
After rotor 3 backward: Y
After rotor 2 backward: V
After rotor 1 backward: M
After final plugboard swap: M
Encrypted character: M

Original message: HELLO WORLD
Encrypted message: MFNDZ AAEZV

Decrypting message:
Initial character: M
After plugboard swap: M
After rotor 1 forward: V
After rotor 2 forward: Y
After rotor 3 forward: Q
After reflector swap: E
After rotor 3 backward: P
After rotor 2 backward: U
After rotor 1 backward: H
After final plugboard swap: H
Encrypted character: H

Encrypted message: MFNDZ AAEZV
Decrypted message: HELLO WORLD


# Stream Cipher : A5/1, RC4

In [9]:
class A5_1:
    def __init__(self, key, frame_number):
        # 초기화
        self.key = key
        self.frame_number = frame_number
        self.R1 = [0] * 19
        self.R2 = [0] * 22
        self.R3 = [0] * 23

        self.initialize_registers()

    def initialize_registers(self):
        # 키와 프레임 번호를 사용하여 레지스터 초기화
        for i in range(64):
            self.clock_all_registers(self.key[i])
        for i in range(22):
            self.clock_all_registers((self.frame_number >> i) & 1)

    def clock_all_registers(self, input_bit):
        # 레지스터 클럭
        self.clock_register(self.R1, [13, 16, 17, 18], input_bit)
        self.clock_register(self.R2, [20, 21], input_bit)
        self.clock_register(self.R3, [7, 20, 21, 22], input_bit)

    def clock_register(self, register, taps, input_bit):
        feedback = input_bit
        for tap in taps:
            feedback ^= register[tap]
        register.pop()
        register.insert(0, feedback)

    def majority_vote(self):
        return (self.R1[8] + self.R2[10] + self.R3[10]) > 1

    def clock_majority(self):
        majority = self.majority_vote()
        if self.R1[8] == majority:
            self.clock_register(self.R1, [13, 16, 17, 18], 0)
        if self.R2[10] == majority:
            self.clock_register(self.R2, [20, 21], 0)
        if self.R3[10] == majority:
            self.clock_register(self.R3, [7, 20, 21, 22], 0)

    def generate_keystream(self, length):
        keystream = []
        for _ in range(length):
            self.clock_majority()
            keystream_bit = self.R1[18] ^ self.R2[21] ^ self.R3[22]
            keystream.append(keystream_bit)
        return keystream

    def encrypt(self, plaintext):
        keystream = self.generate_keystream(len(plaintext))
        ciphertext = [(bit ^ keystream[i]) for i, bit in enumerate(plaintext)]
        return ciphertext

    def decrypt(self, ciphertext):
        return self.encrypt(ciphertext)  # 암호화와 복호화가 동일

# 키와 프레임 번호 예시
key = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
frame_number = 0b0000000000000000000000

# 예시 평문 (비트 배열로)
plaintext = [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0]

a5_1 = A5_1(key, frame_number)

# 암호화
ciphertext = a5_1.encrypt(plaintext)
print(f"Ciphertext: {ciphertext}")

# 복호화
decrypted_plaintext = a5_1.decrypt(ciphertext)
print(f"Decrypted plaintext: {decrypted_plaintext}")


Ciphertext: [1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]
Decrypted plaintext: [0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]


In [10]:
class RC4:
    def __init__(self, key):
        self.key = key
        self.S = self.initialize_S(key)

    def initialize_S(self, key):
        # 초기화 벡터 S와 키 K를 설정
        S = list(range(256))
        K = [ord(c) for c in key]
        j = 0

        # 키 스케줄링 알고리즘(KSA)
        for i in range(256):
            j = (j + S[i] + K[i % len(K)]) % 256
            S[i], S[j] = S[j], S[i]
        
        return S

    def generate_keystream(self, length):
        # 의사 난수 생성 알고리즘(PRGA)
        i = 0
        j = 0
        S = self.S[:]
        keystream = []

        for _ in range(length):
            i = (i + 1) % 256
            j = (j + S[i]) % 256
            S[i], S[j] = S[j], S[i]
            K = S[(S[i] + S[j]) % 256]
            keystream.append(K)
        
        return keystream

    def encrypt(self, plaintext):
        # 평문을 바이트 배열로 변환
        plaintext_bytes = [ord(c) for c in plaintext]
        keystream = self.generate_keystream(len(plaintext_bytes))
        ciphertext = [p ^ k for p, k in zip(plaintext_bytes, keystream)]
        return ciphertext

    def decrypt(self, ciphertext):
        # 암호화와 동일한 과정
        keystream = self.generate_keystream(len(ciphertext))
        plaintext_bytes = [c ^ k for c, k in zip(ciphertext, keystream)]
        plaintext = ''.join(chr(b) for b in plaintext_bytes)
        return plaintext

# 예제 키
key = "SecretKey"
rc4 = RC4(key)

# 예제 평문
plaintext = "Hello, World!"

# 암호화
ciphertext = rc4.encrypt(plaintext)
print(f"Ciphertext: {ciphertext}")

# 복호화
decrypted_plaintext = rc4.decrypt(ciphertext)
print(f"Decrypted plaintext: {decrypted_plaintext}")


Ciphertext: [92, 221, 80, 43, 176, 94, 62, 245, 139, 97, 81, 34, 140]
Decrypted plaintext: Hello, World!


# Data Encryption Standard(DES)

In [4]:
s_boxes = [
    # S1
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7], 
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0], 
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    # S2
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10], 
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15], 
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    # S3
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8], 
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7], 
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    # S4
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15], 
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4], 
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    # S5
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9], 
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14], 
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    # S6
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11], 
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6], 
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    # S7
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1], 
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2], 
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    # S8
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7], 
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8], 
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]
    
def s_box_lookup(input_6bits, s_box_index):
    row = int(input_6bits[0] + input_6bits[5], 2)
    col = int(input_6bits[1:5], 2)
    return format(s_boxes[s_box_index][row][col], '04b')

# Apply each step of the round function and print the results
def DES_round(input_bits, round_key):
    print("Input bits:", input_bits)
    print("Round key:", round_key)
    # Expand the input to 48 bits
    expansion_table = [31, 0, 1, 2, 3, 4,
                       3, 4, 5, 6, 7, 8,
                       7, 8, 9, 10, 11, 12,
                       11, 12, 13, 14, 15, 16,
                       15, 16, 17, 18, 19, 20,
                       19, 20, 21, 22, 23, 24,
                       23, 24, 25, 26, 27, 28,
                       27, 28, 29, 30, 31, 0]
    
    expanded_input = "".join(input_bits[i % len(input_bits)] for i in expansion_table)
    print("After Expansion:", expanded_input)

    
    # XOR with round key
    xor_result = format(int(expanded_input, 2) ^ int(round_key, 2), '048b')
    print("After XOR with round key:", xor_result)
 
    # Apply S-Boxes
    s_box_output = ""
    for i in range(8):
        s_box_output += s_box_lookup(xor_result[i*6:i*6+6], i)
    print("After S-Box substitution:",s_box_output)
    
    # Permutation
    permutation_table = [16, 7, 20, 21, 29, 12, 28, 17,
                         1, 15, 23, 26, 5, 18, 31, 10,
                         2, 8, 24, 14, 32, 27, 3, 9,
                         19, 13, 30, 6, 22, 11, 4, 25]
    output = ""
    for index in permutation_table:
        if index <= 32:
            output += s_box_output[index-1]
    print("After Permutation:", output)
    
# Example usage
input_bits = "01100110011001100110011001010101"
round_key = "000110110000001011101111111111000111000001110010"
DES_round(input_bits, round_key)


Input bits: 01100110011001100110011001010101
Round key: 000110110000001011101111111111000111000001110010
After Expansion: 101100001100001100001100001100001100001010101010
After XOR with round key: 101010111100000111100011110011001011001011011000
After S-Box substitution: 01100010100111111111110010010101
After Permutation: 11110111010001001001101111101001


In [5]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


# AES

In [6]:

from Crypto.Cipher import AES
from Crypto.Util import Padding
import numpy as np

# AES 암호화 객체 생성
def AES_round(input_block, round_key):
    print("Input_block:", input_block)
    print("Round key:", round_key)
    cipher = AES.new(round_key, AES.MODE_ECB)

    # AES 표준 S-박스 테이블
    s_box = [
        0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
        0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
        0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
        0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
        0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
        0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
        0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
        0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
        0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
        0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
        0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
        0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
        0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
        0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
        0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]

    # SubBytes 단계
    def sub_bytes(state):
        return np.array([[s_box[b] for b in row] for row in state], dtype=np.uint8)

  # ShiftRows 단계
    def shift_rows(state):
       shifted_state = np.zeros_like(state)
       for i in range(4):
          shifted_state[i] = np.roll(state[i], -i)
       return shifted_state

    # MixColumns 단계
    def mix_columns(state):
        matrix = np.array([[2, 3, 1, 1],
                           [1, 2, 3, 1],
                           [1, 1, 2, 3],
                           [3, 1, 1, 2]])
        return np.dot(matrix, state) % 0x11b

    # AddRoundKey 단계
    def add_round_key(state, round_key):
        round_key_array = np.frombuffer(round_key, dtype=np.uint8).reshape(4, 4)
        return state ^ round_key_array

    # 입력 블록을 4x4 행렬로 변환
    state = np.array([[input_block[i + j * 4] for i in range(4)] for j in range(4)], dtype=np.uint8)

    # SubBytes 단계 적용
    state = sub_bytes(state)
    print("After SubBytes:")
    print(state)

    # ShiftRows 단계 적용
    state = shift_rows(state)
    print("After ShiftRows:")
    print(state)

    # MixColumns 단계 적용
    state = mix_columns(state)
    print("After MixColumns:")
    print(state)

    # AddRoundKey 단계 적용
    state = add_round_key(state, round_key)
    print("After AddRoundKey:")
    print(state)

# 예제 입력 블록 및 라운드 키
input_block = [0x32, 0x88, 0x31, 0xe0,
               0x43, 0x5a, 0x31, 0x37,
               0xf6, 0x30, 0x98, 0x07,
               0xa8, 0x8d, 0xa2, 0x34]
round_key = b'Sixteen byte key'
# [[ 83 105 120 116] ASCII 코드에서 S는 83, i 는 105
# [101 101 110  32]
# [ 98 121 116 101]
# [ 32 107 101 121]]

# 한 라운드를 수행
AES_round(input_block, round_key)


Input_block: [50, 136, 49, 224, 67, 90, 49, 55, 246, 48, 152, 7, 168, 141, 162, 52]
Round key: b'Sixteen byte key'
After SubBytes:
[[ 35 196 199 225]
 [ 26 190 199 154]
 [ 66   4  70 197]
 [194  93  58  24]]
After ShiftRows:
[[ 35 196 199 225]
 [190 199 154  26]
 [ 70 197  66   4]
 [ 24 194  93  58]]
After MixColumns:
[[168 248 170  24]
 [ 83 247 232  64]
 [154 239 198 150]
 [130 240 154 255]]
After AddRoundKey:
[[251 145 210 108]
 [ 54 146 134  96]
 [248 150 178 243]
 [162 155 255 134]]


# RSA

In [7]:
import sympy
import random

# 소수 선택
def select_primes():
    p = sympy.randprime(2**7, 2**8)
    q = sympy.randprime(2**7, 2**8)
    return p, q

# 모듈러 계산
def calculate_modulus(p, q):
    return p * q

# 오일러 피 함수 계산
def calculate_phi(p, q):
    return (p - 1) * (q - 1)

# 공개키 선택
def select_public_key(phi):
    while True:
        e = random.randint(2, phi - 1)
        if sympy.isprime(e) and sympy.gcd(e, phi) == 1:
            return e

# 비밀키 선택
def select_private_key(e, phi):
    d = sympy.mod_inverse(e, phi)
    return d

# 암호화
def encrypt(message, e, n):
    return pow(message, e, n)

# 복호화
def decrypt(ciphertext, d, n):
    return pow(ciphertext, d, n)

# RSA 암호화 과정
def rsa_process():
    # 소수 선택
    p, q = select_primes()
    print("Selected primes: p =", p, ", q =", q)

    # 모듈러 계산
    n = calculate_modulus(p, q)
    print("Modulus (n):", n)

    # 오일러 피 함수 계산
    phi = calculate_phi(p, q)
    print("Euler's totient (phi):", phi)

    # 공개키 선택
    e = select_public_key(phi)
    print("Public key (e):", e)

    # 비밀키 선택
    d = select_private_key(e, phi)
    print("Private key (d):", d)

    # 메시지 입력
    message = int(input("Enter the message to encrypt: "))
    print("Original message:", message)

    # 암호화
    encrypted_message = encrypt(message, e, n)
    print("Encrypted message:", encrypted_message)

    # 복호화
    decrypted_message = decrypt(encrypted_message, d, n)
    print("Decrypted message:", decrypted_message)

# RSA 암호화 과정 실행
rsa_process()


Selected primes: p = 233 , q = 149
Modulus (n): 34717
Euler's totient (phi): 34336
Public key (e): 28393
Private key (d): 20025
Enter the message to encrypt: 7657
Original message: 7657
Encrypted message: 18548
Decrypted message: 7657


# SHA-1

In [1]:
import struct
import math

# Constants for MD5
s = [7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4
K = [int(abs(math.sin(i + 1)) * (2**32)) & 0xFFFFFFFF for i in range(64)]
init_values = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476]

def left_rotate(x, c):
    return (x << c) & 0xFFFFFFFF | (x >> (32 - c))

def md5(message):
    message = bytearray(message.encode('utf-8'))
    orig_len_in_bits = (8 * len(message)) & 0xFFFFFFFFFFFFFFFF
    message.append(0x80)
    while len(message) % 64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

    A, B, C, D = init_values

    for chunk_ofst in range(0, len(message), 64):
        a, b, c, d = A, B, C, D
        chunk = message[chunk_ofst:chunk_ofst + 64]
        M = struct.unpack('<16I', chunk)

        for i in range(64):
            if 0 <= i <= 15:
                f = (b & c) | (~b & d)
                g = i
            elif 16 <= i <= 31:
                f = (d & b) | (~d & c)
                g = (5 * i + 1) % 16
            elif 32 <= i <= 47:
                f = b ^ c ^ d
                g = (3 * i + 5) % 16
            elif 48 <= i <= 63:
                f = c ^ (b | ~d)
                g = (7 * i) % 16

            f = (f + a + K[i] + M[g]) & 0xFFFFFFFF
            a, d, c, b = d, (b + left_rotate(f, s[i])) & 0xFFFFFFFF, b, c

        A = (A + a) & 0xFFFFFFFF
        B = (B + b) & 0xFFFFFFFF
        C = (C + c) & 0xFFFFFFFF
        D = (D + d) & 0xFFFFFFFF

    return '{:08x}{:08x}{:08x}{:08x}'.format(A, B, C, D)

# Example usage
message = "The quick brown fox jumps over the lazy dog"
hash_value = md5(message)
print(f"MD5('{message}') = {hash_value}")


MD5('The quick brown fox jumps over the lazy dog') = 9d96ec89df9b57123175b9fc6d6e9599
